In [ ]:
!pip install mlflow torchmetrics torchinfo --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 28.2/28.2 MB 75.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.1/6.1 MB 109.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 960.9/960.9 kB 56.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 231.9/231.9 kB 21.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.8/147.8 kB 13.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.9/114.9 kB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.0/85.0 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 114.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 87.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 49.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 877.0 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
!pip install GPUtil

  Preparing metadata (setup.py) ... done
  Created wheel for GPUtil: filename=GPUtil-1.4.0-py3-none-any.whl size=7392 sha256=68982009b2fa11a8861dda5b69f52a6dc9e5dfff878a41c8091e0daf5ea8ca84
  Stored in directory: /root/.cache/pip/wheels/2b/4d/8f/55fb4f7b9b591891e8d3f72977c4ec6c7763b39c19f0861595
Successfully built GPUtil


In [ ]:
import os
import shutil
import pprint

from random import random, randint
from mlflow import log_metric, log_param, log_artifacts
from mlflow.tracking import MlflowClient
import warnings

import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader
from torchinfo import summary
from torchmetrics import Accuracy, F1Score
from torchvision import datasets
from torchvision.transforms import ToTensor

import mlflow
import pandas as pd
from sklearn.preprocessing import LabelEncoder
import torchvision.transforms as transforms
from PIL import Image
import numpy as np
import torch.optim as optim
import time
from sklearn.metrics import f1_score
import torch.nn.functional as F
import tensorflow as tf
import GPUtil


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
warnings.filterwarnings("ignore")
print(mlflow.__version__)

2.21.2


In [ ]:
warnings.filterwarnings("ignore")
print(mlflow.__version__)

2.21.2


In [ ]:
folder_path = '/content/drive/MyDrive/ap_proj1'
files_in_folder = os.listdir(folder_path + "/data")
print(files_in_folder)
print(os.getcwd())

['train_labels.csv', 'Test', 'Train', 'TrainCropped']
/content


In [ ]:
labels = pd.read_csv(folder_path + "/data/train_labels.csv")
image_folder = folder_path + "/data/TrainCropped/"

In [ ]:
label_encoder = LabelEncoder()
labels["label"] = label_encoder.fit_transform(labels["label"])

In [ ]:
num_classes = len(label_encoder.classes_)
print(num_classes)

9


In [ ]:
get_ipython().system_raw("mlflow ui --backend-store-uri sqlite:///drive/MyDrive/ap_proj1/mlflowdata/mlruns.db --port 5000 &")# run tracking UI in the background
local_registry = "sqlite:///drive/MyDrive/ap_proj1/mlflowdata/mlruns.db"
model_name="MLP"
mlflow.set_tracking_uri(local_registry)
mlflow.set_experiment("/task1")

2025/03/31 12:00:18 INFO mlflow.tracking.fluent: Experiment with name '/task1' does not exist. Creating a new experiment.


<Experiment: artifact_location='/content/mlruns/2', creation_time=1743422418043, experiment_id='2', last_update_time=1743422418043, lifecycle_stage='active', name='/task1', tags={}>

In [3]:
from pyngrok import ngrok

In [5]:
ngrok.kill()

In [4]:
NGROK_AUTH_TOKEN = "2uryxEZ5QrQQ0SGn8gTWLAAT1ei_7Njezw5fQYwuwdb4mfhpk" # <- Paste your token here
ngrok.set_auth_token(NGROK_AUTH_TOKEN)

In [ ]:
public_url = ngrok.connect("5000")
print("MLflow Tracking UI:", public_url)

MLflow Tracking UI: NgrokTunnel: "https://7f6a-35-203-130-43.ngrok-free.app" -> "http://localhost:5000"


In [ ]:
def crop_pokemon(img_path):
    img = Image.open(img_path)
    gray_img = img.convert("L") # Convert to grayscale
    img_array = np.array(gray_img)

    # Detect black silhouette (thresholding)
    threshold = 10
    mask = img_array < threshold

    # Get coordinates of silhouette
    coords = np.column_stack(np.where(mask))
    if coords.shape[0] == 0:
        return img.resize((64, 64))  # If no silhouette found, return resized image

    y_min, x_min = coords.min(axis=0)
    y_max, x_max = coords.max(axis=0)

    # Apply padding (5%)
    pad = int(0.05 * min(x_max - x_min, y_max - y_min))
    x_min, y_min = max(0, x_min - pad), max(0, y_min - pad)
    x_max, y_max = min(img.width, x_max + pad), min(img.height, y_max + pad)

    cropped_img = img.crop((x_min, y_min, x_max, y_max))
    cropped_img = cropped_img.resize((64, 64))

    return cropped_img

In [ ]:
class PokemonDataset(Dataset):
    def __init__(self, df, img_dir, transform=None):
        self.df = df
        self.img_dir = img_dir
        self.transform = transform

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        img_name = os.path.join(self.img_dir, str(self.df.iloc[idx, 0]) + ".png")

        #image = crop_pokemon(img_name)

        if self.transform:
            image = self.transform(Image.open(img_name))

        if len(self.df.columns) > 1:  # Train Set has labels, Test does not.
            label = self.df.iloc[idx, 1]
            return image, label
        else:
            return image, -1  # X dont care for Test

In [ ]:
# Define MLP Model
class MLP(nn.Module):
    def __init__(self, input_size, num_classes, layer1_size = 512, layer2_size = 256, dropout = 0.3):
        super(MLP, self).__init__()
        self.fc1 = nn.Linear(input_size, layer1_size)
        self.bn1 = nn.BatchNorm1d(layer1_size)
        self.fc2 = nn.Linear(layer1_size, layer2_size)
        self.bn2 = nn.BatchNorm1d(layer2_size)
        self.fc3 = nn.Linear(layer2_size, num_classes)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        x = F.relu(self.bn1(self.fc1(x)))
        x = self.dropout(x)
        x = F.relu(self.bn2(self.fc2(x)))
        x = self.fc3(x)
        return x

In [ ]:
# Define transformations for better generalization
transform = transforms.Compose([
    transforms.RandomHorizontalFlip(), # Random flipping left-right
    transforms.RandomRotation(15), # Rotate randomly within ±15 degrees
    transforms.ColorJitter(brightness=0.2, contrast=0.2), # Adjust brightness/contrast
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,)),  # Normalize
    transforms.Lambda(lambda x: x.view(-1))  # Flatten
])

In [ ]:
# Split the dataset into 3 (70% train, 10% val, 20% test)
dataset = PokemonDataset(labels, image_folder, transform=transform)
train_size = int(0.7 * len(dataset))
val_size = int(0.15 * len(dataset))
test_size = len(dataset) - train_size - val_size
train_dataset, val_dataset, test_dataset = torch.utils.data.random_split(dataset, [train_size, val_size, test_size])

In [ ]:
sample_image, _ = dataset[0]
input_size = sample_image.numel()

In [ ]:
# Choose single hyperparameter values
n_epochs = 50
batch_size = 16
learning_rate = 0.001
step_size = 5
l1_size = 256
l2_size = 128
dropout = 0.2
gamma = 0.1

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [ ]:
# Create data loaders
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [ ]:
# Initialize model, loss function, optimizer, and scheduler
model = MLP(input_size, num_classes, l1_size, l2_size, dropout)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=step_size, gamma=gamma)
pip_requirements = ["torch==2.6.0+cu124"]
model.to(device)

MLP(
  (fc1): Linear(in_features=16384, out_features=256, bias=True)
  (bn1): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc2): Linear(in_features=256, out_features=128, bias=True)
  (bn2): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc3): Linear(in_features=128, out_features=9, bias=True)
  (dropout): Dropout(p=0.2, inplace=False)
)

In [ ]:
def log_gpu_usage():
    gpus = GPUtil.getGPUs()
    for i, gpu in enumerate(gpus):
        mlflow.log_metric(f"gpu_{i}_memory_used_MB", gpu.memoryUsed)
        mlflow.log_metric(f"gpu_{i}_memory_total_MB", gpu.memoryTotal)
        mlflow.log_metric(f"gpu_{i}_utilization_percent", gpu.load * 100)


In [ ]:
# Training Loop
def training_loop():
    start_time = time.time()

    # Training loop
    for epoch in range(n_epochs):
      model.train()
      running_loss = 0.0
      for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

        log_gpu_usage()

      # Validation loop
      model.eval()
      all_labels = []
      all_preds = []
      with torch.no_grad():
        for images, labels in val_loader:
          images, labels = images.to(device), labels.to(device)
          outputs = model(images)
          _, preds = torch.max(outputs, 1)  # Get predicted class indices
          all_labels.extend(labels.cpu().numpy())
          all_preds.extend(preds.cpu().numpy())

      # Calculate F1 Score
      f1 = f1_score(all_labels, all_preds, average="macro")
      print(f"Epoch [{epoch+1}/{n_epochs}], Loss: {running_loss/len(train_loader):.4f}, F1 Score: {f1:.4f}")
      mlflow.log_metric("loss", f"{running_loss/len(train_loader):.4f}")
      mlflow.log_metric("f1_score", f"{f1:.4f}")

    end_time = time.time()
    execution_time = end_time - start_time
    print(f"Execution time: {execution_time} seconds")

    # Test loop
    model.eval()
    correct, total = 0, 0
    all_labels = []
    all_preds = []
    with torch.no_grad():
      for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        all_labels.extend(labels.cpu().numpy())
        all_preds.extend(predicted.cpu().numpy())

    accuracy = 100 * correct / total
    f1 = f1_score(all_labels, all_preds, average="macro")
    mlflow.log_metric("eval_loss", f"{running_loss/len(train_loader):.4f}")
    mlflow.log_metric("eval_accuracy", f"{f1:.4f}")
    print(f"Test Accuracy: {accuracy:.2f}%")
    print(f"Test F1 Score: {f1:.4f}")

In [ ]:
with mlflow.start_run() as run:
  params = {
        "epochs": n_epochs,
        "learning_rate": learning_rate,
        "batch_size": batch_size,
        "loss_function": criterion.__class__.__name__,
        "metric_function": f1_score.__name__,
        "optimizer": "Adam",
        "step_size": step_size,
        "layer1_size": l1_size,
        "layer2_size": l2_size,
        "dropout": dropout,
        "gamma": gamma
    }
  output_folder = os.path.join(folder_path, "outputs")
  if not os.path.exists(output_folder):
      os.makedirs(output_folder)

  # Log training parameters.
  mlflow.log_params(params)

  # Log model summary.
  with open(os.path.join(output_folder, "model_summary.txt"), "w") as f:
      f.write(str(summary(model)))

  log_artifacts(output_folder)
  mlflow.log_artifact(os.path.join(output_folder, "model_summary.txt"))

  training_loop()

  mlflow.pytorch.log_model(model, "model", pip_requirements=pip_requirements)#, input_example=sample_image.numpy())

Epoch [1/50], Loss: 1.7945, F1 Score: 0.6038
Epoch [2/50], Loss: 1.1560, F1 Score: 0.7473
Epoch [3/50], Loss: 0.8610, F1 Score: 0.8037
Epoch [4/50], Loss: 0.7759, F1 Score: 0.8614
Epoch [5/50], Loss: 0.6254, F1 Score: 0.8584
Epoch [6/50], Loss: 0.6012, F1 Score: 0.8861
Epoch [7/50], Loss: 0.5411, F1 Score: 0.9050
Epoch [8/50], Loss: 0.4887, F1 Score: 0.8796
Epoch [9/50], Loss: 0.4683, F1 Score: 0.8832
Epoch [10/50], Loss: 0.4411, F1 Score: 0.8971
Epoch [11/50], Loss: 0.4448, F1 Score: 0.9051
Epoch [12/50], Loss: 0.4428, F1 Score: 0.8965
Epoch [13/50], Loss: 0.4110, F1 Score: 0.9081
Epoch [14/50], Loss: 0.4131, F1 Score: 0.9029
Epoch [15/50], Loss: 0.3781, F1 Score: 0.9110
Epoch [16/50], Loss: 0.3888, F1 Score: 0.9188
Epoch [17/50], Loss: 0.3914, F1 Score: 0.9167
Epoch [18/50], Loss: 0.3534, F1 Score: 0.9011
Epoch [19/50], Loss: 0.3478, F1 Score: 0.9026
Epoch [20/50], Loss: 0.3580, F1 Score: 0.9149
Epoch [21/50], Loss: 0.3539, F1 Score: 0.9111
Epoch [22/50], Loss: 0.3297, F1 Score: 0.90

2025/03/31 13:08:25 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


In [ ]:
client = MlflowClient()

print("List of all registered models")
print("=" * 80)
[print(pprint.pprint(dict(rm), indent=4)) for rm in client.search_registered_models()]

# Get a list of specific versions of the named models
print(f"\nList of Model = {model_name} and Versions")
print("=" * 80)
[pprint.pprint(dict(mv), indent=4) for mv in client.search_model_versions()]

List of all registered models
{   'aliases': {},
    'creation_timestamp': 1743092803691,
    'description': None,
    'last_updated_timestamp': 1743092803751,
    'latest_versions': [   <ModelVersion: aliases=[], creation_timestamp=1743092803751, current_stage='None', description=None, last_updated_timestamp=1743092803751, name='MLP', run_id='b2e8bf02d84647068e8b6324abad84bf', run_link=None, source='/content/mlruns/1/b2e8bf02d84647068e8b6324abad84bf/artifacts/sklearn-model', status='READY', status_message=None, tags={}, user_id=None, version=1>],
    'name': 'MLP',
    'tags': {}}
None

List of Model = MLP and Versions
{   'aliases': [],
    'creation_timestamp': 1743092803751,
    'current_stage': 'None',
    'description': None,
    'last_updated_timestamp': 1743092803751,
    'name': 'MLP',
    'run_id': 'b2e8bf02d84647068e8b6324abad84bf',
    'run_link': None,
    'source': '/content/mlruns/1/b2e8bf02d84647068e8b6324abad84bf/artifacts/sklearn-model',
    'status': 'READY',
    'st

[None]